# Main notebook for model train and tracking

In [1]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
import pandas as pd
import numpy as np 
from sklearn.model_selection import KFold, train_test_split
import matplotlib.pyplot as plt
import mlflow
import xgboost as xgb

In [3]:
cat_cols = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']

train = pd.read_parquet("../data/train_woe_balanced.parquet")
train = train[:10000]

In [9]:
train.drop('WOE_target',axis=1, inplace=True)

In [23]:
FEATURES = train.columns.to_list()
FEATURES.remove("target")
FEATURES = FEATURES[2:]

227

In [24]:
#Competition metric
def amex_metric(y_true, y_pred):

    labels     = np.transpose(np.array([y_true, y_pred]))
    labels     = labels[labels[:, 1].argsort()[::-1]]
    weights    = np.where(labels[:,0]==0, 20, 1)
    cut_vals   = labels[np.cumsum(weights) <= int(0.04 * np.sum(weights))]
    top_four   = np.sum(cut_vals[:,0]) / np.sum(labels[:,0])

    gini = [0,0]
    for i in [1,0]:
        labels         = np.transpose(np.array([y_true, y_pred]))
        labels         = labels[labels[:, i].argsort()[::-1]]
        weight         = np.where(labels[:,0]==0, 20, 1)
        weight_random  = np.cumsum(weight / np.sum(weight))
        total_pos      = np.sum(labels[:, 0] *  weight)
        cum_pos_found  = np.cumsum(labels[:, 0] * weight)
        lorentz        = cum_pos_found / total_pos
        gini[i]        = np.sum((lorentz - weight_random) * weight)

    return 0.5 * (gini[1]/gini[0] + top_four)

In [27]:
## Model Parameters

# xgb_params = {
#     'max_depth':4,
#     'booster': 'dart',
#     'alpha': 0.1,
#     'learning_rate':0.035,
#     'subsample': 0.8,
#     'colsample_bytree':0.6,
#     'eval_metric': 'logloss',
#     'objective':'binary:logistic',
#     'tree_method':'gpu_hist',
#     'predictor':'gpu_predictor',
#     'random_state':42
# }

xgb_params = {
        'num_leaves': 10,
        'max_bin': 127,
        'min_data_in_leaf': 11,
        'learning_rate': 0.02,
        'min_sum_hessian_in_leaf': 0.00245,
        'bagging_fraction': 1.0, 
        'bagging_freq': 5, 
        'feature_fraction': 0.05,
        'lambda_l1': 4.972,
        'lambda_l2': 2.276,
        'min_gain_to_split': 0.65,
        'max_depth': 14,
        'save_binary': True,
        'seed': 1337,
        'feature_fraction_seed': 1337,
        'bagging_seed': 1337,
        'drop_seed': 1337,
        'data_random_seed': 1337,
        'objective': 'binary',
        'boosting_type': 'dart',
        'verbose': 1,
        'is_unbalance': True,
        'boost_from_average': False,
        'device': 'gpu',
        'gpu_platform_id': 0,
        'gpu_device_id': 0
        }

reg_log_params={
        'penalty': 'l2',
        'max_iter': 200,
        'warm_start': True,
        'n_jobs': 1,
            
                }
fores_params={
        'bootstrap': True,
        'criterion': 'mse',
        'max_depth': None,
        'max_features': 'auto',
        'max_leaf_nodes': None,
        'min_impurity_decrease': 0.0,
        'min_impurity_split': None,
        'min_samples_leaf': 1,
        'min_samples_split': 2,
        'min_weight_fraction_leaf': 0.0,
        'n_estimators': 10,
        'n_jobs': -1,
        'oob_score': False,
        'random_state': 42,
        'verbose': 0,
        'warm_start': False
            }

In [33]:
models= {
        "reg_log":LogisticRegression(reg_log_params),
        "xgb": xgb.XGBClassifier(xgb_params),
        "forest": RandomForestClassifier(fores_params)
}

# VERSION NAME FOR SAVED MODEL FILES
VER = '03'

# TRAIN RANDOM SEED
SEED = 42

# FILL NAN VALUE
NAN_VALUE = -127 # will fit in int8

# FOLDS PER MODEL
FOLDS = 5

# TRAIN FOLD
#TRAIN_PATH = "../data/processed/train_woe_balanced.parquet"

importances = []
oof = []
TRAIN_SUBSAMPLE = 1.0

skf = KFold(n_splits = FOLDS, shuffle=True, random_state=42)

for fold, (train_idx, valid_idx) in enumerate(skf.split(train, train.target)):
        for k in models.v:



In [34]:
for k in models.values():
    print(k)

LogisticRegression(penalty={'max_iter': 200, 'n_jobs': 1, 'penalty': 'l2',
                            'warm_start': True})
XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, gamma=None,
              gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_to_onehot=None, max_delta_s...
                         'feature_fraction_seed': 1337, 'gpu_device_id': 0,
                         'gpu_platform_id': 0, 'is_unbalance': True,
                         'lambda_l1': 4.972, 'lambda_l2': 2.276,
                         'learning_rate': 0.02, 'max_bin': 127, 'max_depth': 14,
                         'min_data_in_leaf': 11, 'min_gain_to_split': 0.65,
                         'min_sum_hessian_in_leaf': 0.